In [1]:
from sqlite3 import connect
import numpy as np
import pandas as pd
import os
import re

In [ ]:
os.getcwd()

In [3]:
conditions_path = '../data/raw/synthea/scenario1/conditions.csv'
encounters_path = '../data/raw/synthea/scenario1/encounters.csv'
patients_path = '../data/raw/synthea/scenario1/patients.csv'
procedures_path = '../data/raw/synthea/scenario1/procedures.csv'

In [4]:
conn = connect(':memory:')

In [5]:
patients_df = pd.read_csv(patients_path)
patients_df.to_sql('patients', conn, if_exists='replace')
patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,7b3c738d-3f86-58e3-450e-4018521d192f,2021-08-11,NaN,999-71-7790,NaN,NaN,NaN,Svetlana462,O'Hara248,NaN,...,Boston Massachusetts US,971 Ullrich Grove Suite 53,Boston,Massachusetts,Suffolk County,2135.0,42.304260,-71.045262,22950.9100,0.0000
1,c40c2c75-13c9-8e4a-047f-573ae1330157,2020-03-03,NaN,999-49-5505,NaN,NaN,NaN,Rosalyn434,Christiansen251,NaN,...,Lawrence Massachusetts US,485 Senger Route Apt 34,Waltham,Massachusetts,Middlesex County,2451.0,42.340439,-71.206815,54769.7275,2624.1225
2,39e76039-522c-61f1-d961-e03dce5f0bb2,1998-11-21,2003-07-17,999-36-5991,NaN,NaN,NaN,Lashawnda573,Daniel959,NaN,...,North Reading Massachusetts US,944 Witting Passage,Everett,Massachusetts,Middlesex County,2149.0,42.412900,-71.009282,139148.2750,932.7450
3,ff981e00-4004-44b4-48ba-0bfa62440bb3,2009-07-29,NaN,999-39-5303,NaN,NaN,NaN,Ulysses632,Rice937,NaN,...,Scituate Massachusetts US,414 Rempel Harbor,Cohasset,Massachusetts,Norfolk County,NaN,42.214770,-70.814553,339359.4700,90.6000
4,648dbf91-4334-a7e2-2cfe-72e88dbd4c15,2000-12-04,NaN,999-77-8366,S99996514,X32384432X,Mr.,Tory770,Harvey63,NaN,...,Boston Massachusetts US,368 Pouros Ramp Apt 46,Haverhill,Massachusetts,Essex County,1832.0,42.779225,-71.055842,543267.4075,2173.6425


In [6]:
conds_df = pd.read_csv(conditions_path)
conds_df.to_sql('conditions', conn, if_exists='replace')
conds_df.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2021-12-26,2022-01-07,c40c2c75-13c9-8e4a-047f-573ae1330157,862287d1-a2eb-8116-de42-e76de6163260,195662009,Acute viral pharyngitis (disorder)
1,1999-05-12,NaN,39e76039-522c-61f1-d961-e03dce5f0bb2,00681d7e-e9bb-392b-6ccb-209cf46e2ec3,128613002,Seizure disorder
2,1999-05-12,NaN,39e76039-522c-61f1-d961-e03dce5f0bb2,00681d7e-e9bb-392b-6ccb-209cf46e2ec3,703151001,History of single seizure (situation)
3,2003-03-04,2003-03-17,39e76039-522c-61f1-d961-e03dce5f0bb2,7ac71954-b14b-f259-f032-bc07d5f4eaa6,43878008,Streptococcal sore throat (disorder)
4,2003-07-17,NaN,39e76039-522c-61f1-d961-e03dce5f0bb2,705cabcc-6c94-1804-ee22-0e0b944ed62f,262574004,Bullet wound


In [7]:
conds_df[['CODE', 'DESCRIPTION']].value_counts().to_csv('../data/interim/scenario1/conditions_type_counts.csv')

In [8]:
pd.read_sql(
    '''
    SELECT * FROM conditions
    WHERE DESCRIPTION LIKE '%drug%' OR DESCRIPTION LIKE '%alcohol%' OR DESCRIPTION LIKE '%opioid%' OR DESCRIPTION LIKE '%tobacco%'
    '''
    , conn
)

,index,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,238,1997-06-03,1999-06-15,04e9ac90-2279-d293-523f-838b39d1e177,20917154-d6c5-6c59-6532-26aecd1165aa,10939881000119104,Unhealthy alcohol drinking behavior (finding)
1,292,2020-09-22,2021-09-28,5f2e1751-1689-2b18-f3df-1cf4a05b5439,e7d4fd73-34ae-a5f3-16a5-4c9ee897f8e9,10939881000119104,Unhealthy alcohol drinking behavior (finding)
2,711,2002-12-27,2014-02-14,53b07e50-426d-43bb-42d9-d175c2fb29f6,2b6ffb87-d675-a040-17e4-61d4a385af70,361055000,Misuses drugs (finding)
3,766,2022-04-01,None,53b07e50-426d-43bb-42d9-d175c2fb29f6,7fc3b70a-fbb7-1b7d-6cc1-8ccf00a2da18,10939881000119104,Unhealthy alcohol drinking behavior (finding)
4,789,2021-11-26,None,ea3c8180-450a-de48-559b-9d753d346507,3d40b5e1-2b58-b08b-5142-0dcc98c1147c,10939881000119104,Unhealthy alcohol drinking behavior (finding)
...,...,...,...,...,...,...,...
351,36139,1952-06-30,None,c20cd05e-a058-b63b-aa3d-38325afdd690,6b443522-a92c-286c-4058-61b0bfc5c488,5602001,Opioid abuse (disorder)
352,36140,1952-06-30,None,c20cd05e-a058-b63b-aa3d-38325afdd690,6b443522-a92c-286c-4058-61b0bfc5c488,449868002,Smokes tobacco daily
353,36241,2005-09-27,2009-10-20,2fd7bfe7-8c26-8be1-b074-efd95829f978,c684d175-c5e0-da6d-5946-147ef37623a1,10939881000119104,Unhealthy alcohol drinking behavior (finding)
354,36285,1995-08-01,1997-08-12,b7fda58e-8eb1-a2d0-8fbf-57afefce34c6,051bc787-cf22-09b9-bc4e-3a6f86c877b6,10939881000119104,Unhealthy alcohol drinking behavior (finding)


In [9]:
pd.read_sql(
    '''
    SELECT * FROM conditions
    WHERE DESCRIPTION LIKE '%lung%'
    '''
    , conn
)

,index,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,4129,1998-09-18,None,f0f321af-2b86-0e6f-de91-338279526b1d,c429e617-1318-4a48-1b27-a0958392f073,162573006,Suspected lung cancer (situation)
1,4130,1998-09-29,None,f0f321af-2b86-0e6f-de91-338279526b1d,8afae70c-cfa6-3211-c050-2cdf8bbfe4b1,254637007,Non-small cell lung cancer (disorder)
2,4131,1998-10-01,None,f0f321af-2b86-0e6f-de91-338279526b1d,071deb68-d763-535a-7d3e-546ec16bee01,424132000,Non-small cell carcinoma of lung TNM stage 1 ...
3,8080,1992-11-20,None,958c3318-1cfe-1122-f4a3-bcf0fabaed05,4be89d3b-ba2f-5f22-b436-9e86ea670b9c,162573006,Suspected lung cancer (situation)
4,8081,1992-12-01,None,958c3318-1cfe-1122-f4a3-bcf0fabaed05,98989e2e-2dc0-6f73-1574-6b96578aa45f,254637007,Non-small cell lung cancer (disorder)
5,8082,1992-12-02,None,958c3318-1cfe-1122-f4a3-bcf0fabaed05,7bb9637a-25d2-53b9-0fbc-8f1bf10dee87,424132000,Non-small cell carcinoma of lung TNM stage 1 ...
6,14561,2016-05-24,None,4e3e9e9c-ab95-c628-6fb6-ddecbd25ac95,d4bb0c88-5b8c-2340-ff4f-69e0692dc7a8,162573006,Suspected lung cancer (situation)
7,14562,2016-06-03,None,4e3e9e9c-ab95-c628-6fb6-ddecbd25ac95,c3773393-bcea-98f2-7f62-c235a1b7ab00,254637007,Non-small cell lung cancer (disorder)
8,14563,2016-06-06,None,4e3e9e9c-ab95-c628-6fb6-ddecbd25ac95,b87803bd-e8d6-f86c-9daf-2e5b3f72cc69,424132000,Non-small cell carcinoma of lung TNM stage 1 ...
9,15812,1987-08-11,None,bfa71154-31e9-c514-2fbc-61c5ee112549,e8b55758-1d9f-5aa2-8fef-fd5abe83d9a9,162573006,Suspected lung cancer (situation)


In [30]:
def match_substring(sub_str, s):
    return sub_str.lower() in s.lower()

conds_df['OPIOID'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('opioid', x))
conds_df['ALCOHOL'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('alcohol', x))
conds_df['MISUSE'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('misuse', x))
conds_df['OVERDOSE'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('overdose', x))
conds_df['TOBACCO'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('tobacco', x))
conds_df['VIOLENCE'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('violence', x))
conds_df['PARTNER'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('partner abuse', x))
conds_df['DEPRESSION'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('depression', x))
conds_df['ANXIETY'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('anxiety', x))
conds_df['STRESS'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('stress', x))

drug_conds_df = conds_df.sort_values(by='START').groupby('PATIENT').aggregate({
    'OPIOID': lambda x: np.sum(x)>0,
    'ALCOHOL': lambda x: np.sum(x)>0,
    'MISUSE': lambda x: np.sum(x)>0,
    'OVERDOSE': lambda x: np.sum(x),
    'TOBACCO': lambda x: np.sum(x)>0,
    'VIOLENCE': lambda x: np.sum(x),
    'PARTNER': lambda x: np.sum(x),
    'DEPRESSION': lambda x: np.sum(x)>0,
    'ANXIETY': lambda x: np.sum(x)>0,
    'STRESS': lambda x: np.sum(x)>0,
})
drug_conds_df.to_sql('drug_conditions', conn, if_exists='replace')
drug_conds_df



,OPIOID,ALCOHOL,MISUSE,OVERDOSE,TOBACCO,VIOLENCE,PARTNER,DEPRESSION,ANXIETY,STRESS
PATIENT,,,,,,,,,,
0000c650-47ba-eaf7-c2e6-ee06774a3d95,False,False,False,0,False,0,0,False,False,True
009c30e1-5fa4-f50c-dfa5-bf3092116bbb,False,False,False,0,False,0,1,False,False,True
00c2e638-e098-f9ff-3610-5af6f565616d,False,False,False,0,False,0,0,False,False,False
00fe47e4-40df-207e-e357-789d2aca44c1,False,False,False,0,False,0,0,False,False,True
014e204c-bd87-a7fd-0ee5-86296391a44d,False,False,False,0,False,0,0,False,False,True
...,...,...,...,...,...,...,...,...,...,...
fdaa221b-c4ee-7609-2cfc-1f034b15b3ef,False,False,False,0,False,0,0,False,True,True
fe0b3504-c5df-4691-055d-34ad7002a5d1,False,False,False,1,False,0,1,False,False,True
ff34b909-0ce6-1d5e-6b6d-9b96c75f08bb,False,False,False,0,False,2,2,False,False,True


In [29]:
drug_conds_df.max(0)

OPIOID        True
ALCOHOL       True
MISUSE        True
OVERDOSE         1
TOBACCO       True
VIOLENCE         7
PARTNER         10
DEPRESSION    True
ANXIETY       True
STRESS        True
dtype: object

In [11]:
def match_substring(sub_str, s):
    return sub_str.lower() in s.lower()

conds_df['LUNG'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('lung', x))

lung_conds_df = conds_df.sort_values(by='START').groupby('PATIENT').aggregate({
    'LUNG': lambda x: np.sum(x)>0,
})
lung_conds_df.to_sql('lung_conditions', conn, if_exists='replace')
lung_conds_df.head()



,LUNG
PATIENT,
0000c650-47ba-eaf7-c2e6-ee06774a3d95,False
009c30e1-5fa4-f50c-dfa5-bf3092116bbb,False
00c2e638-e098-f9ff-3610-5af6f565616d,False
00fe47e4-40df-207e-e357-789d2aca44c1,False
014e204c-bd87-a7fd-0ee5-86296391a44d,False


In [12]:
def match_substring(sub_str, s):
    return sub_str.lower() in s.lower()

conds_df['DEPRESSION'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('depression', x))
conds_df['ANXIETY'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('anxiety', x))
conds_df['STRESS'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('stress', x))

depression_conds_df = conds_df.sort_values(by='START').groupby('PATIENT').aggregate({
    'DEPRESSION': lambda x: np.sum(x)>0,
    'ANXIETY': lambda x: np.sum(x)>0,
    'STRESS': lambda x: np.sum(x)>0,
})
depression_conds_df.to_sql('depression_conditions', conn, if_exists='replace')
depression_conds_df.head()



,DEPRESSION,ANXIETY,STRESS
PATIENT,,,
0000c650-47ba-eaf7-c2e6-ee06774a3d95,False,False,True
009c30e1-5fa4-f50c-dfa5-bf3092116bbb,False,False,True
00c2e638-e098-f9ff-3610-5af6f565616d,False,False,False
00fe47e4-40df-207e-e357-789d2aca44c1,False,False,True
014e204c-bd87-a7fd-0ee5-86296391a44d,False,False,True


In [13]:
def match_substring(sub_str, s):
    return sub_str.lower() in s.lower()

conds_df['ARREST'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('Cardiac Arrest', x))
conds_df['INFARCTION'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('Myocardial Infarction', x))

cardiac_conds_df = conds_df.sort_values(by='START').groupby('PATIENT').aggregate({
    'ARREST': lambda x: np.sum(x)>0,
    'INFARCTION': lambda x: np.sum(x)>0,
})
cardiac_conds_df.to_sql('cardiac_conditions', conn, if_exists='replace')
cardiac_conds_df.head()



,ARREST,INFARCTION
PATIENT,,
0000c650-47ba-eaf7-c2e6-ee06774a3d95,False,False
009c30e1-5fa4-f50c-dfa5-bf3092116bbb,False,False
00c2e638-e098-f9ff-3610-5af6f565616d,False,False
00fe47e4-40df-207e-e357-789d2aca44c1,False,False
014e204c-bd87-a7fd-0ee5-86296391a44d,False,False


In [14]:
def match_substring(sub_str, s):
    return sub_str.lower() in s.lower()

conds_df['CONCUSSION'] = conds_df['DESCRIPTION'].apply(lambda x: match_substring('concussion', x))

concussion_conds_df = conds_df.sort_values(by='START').groupby('PATIENT').aggregate({
    'CONCUSSION': lambda x: np.sum(x)>0,
})
concussion_conds_df.to_sql('concussion_conditions', conn, if_exists='replace')
concussion_conds_df.head()



,CONCUSSION
PATIENT,
0000c650-47ba-eaf7-c2e6-ee06774a3d95,False
009c30e1-5fa4-f50c-dfa5-bf3092116bbb,False
00c2e638-e098-f9ff-3610-5af6f565616d,False
00fe47e4-40df-207e-e357-789d2aca44c1,False
014e204c-bd87-a7fd-0ee5-86296391a44d,False


In [35]:
patient_drugs = pd.read_sql(
    '''
    SELECT p.Id, p.birthdate, p.deathdate, d.OPIOID, d.ALCOHOL, d.MISUSE, d.OVERDOSE, d.TOBACCO, d.VIOLENCE, d.PARTNER, d.DEPRESSION, d.ANXIETY, d.STRESS
    FROM patients p
    JOIN drug_conditions d on p.Id = d.patient
    ''',
    conn
)
patient_drugs[~pd.isna(patient_drugs['DEATHDATE'])].to_csv('../data/interim/scenario1/patient-drugs.csv')

In [16]:
patient_lungs = pd.read_sql(
    '''
    SELECT p.Id, p.birthdate, p.deathdate, l.LUNG
    FROM patients p
    JOIN lung_conditions l on p.Id = l.patient
    ''',
    conn
)
patient_lungs[~pd.isna(patient_lungs['DEATHDATE'])].sum(0)

Id           39e76039-522c-61f1-d961-e03dce5f0bb20d679725-4...
BIRTHDATE    1998-11-211973-04-201960-12-271973-04-201952-0...
DEATHDATE    2003-07-172005-08-192000-06-212011-12-161955-0...
LUNG                                                        13
dtype: object

In [17]:
patient_depressions = pd.read_sql(
    '''
    SELECT p.Id, p.birthdate, p.deathdate, l.DEPRESSION, l.ANXIETY, l.STRESS
    FROM patients p
    JOIN depression_conditions l on p.Id = l.patient
    ''',
    conn
)
patient_depressions[~pd.isna(patient_depressions['DEATHDATE'])].sum(0)

Id            39e76039-522c-61f1-d961-e03dce5f0bb20d679725-4...
BIRTHDATE     1998-11-211973-04-201960-12-271973-04-201952-0...
DEATHDATE     2003-07-172005-08-192000-06-212011-12-161955-0...
DEPRESSION                                                    0
ANXIETY                                                      15
STRESS                                                      128
dtype: object

In [18]:
patient_cardiacs = pd.read_sql(
    '''
    SELECT p.Id, p.birthdate, p.deathdate, l.ARREST, l.INFARCTION
    FROM patients p
    JOIN cardiac_conditions l on p.Id = l.patient
    ''',
    conn
)
patient_cardiacs[~pd.isna(patient_cardiacs['DEATHDATE'])].sum(0)

Id            39e76039-522c-61f1-d961-e03dce5f0bb20d679725-4...
BIRTHDATE     1998-11-211973-04-201960-12-271973-04-201952-0...
DEATHDATE     2003-07-172005-08-192000-06-212011-12-161955-0...
ARREST                                                       15
INFARCTION                                                   25
dtype: object

In [20]:
patient_concussions = pd.read_sql(
    '''
    SELECT p.Id, p.birthdate, p.deathdate, l.CONCUSSION
    FROM patients p
    JOIN concussion_conditions l on p.Id = l.patient
    ''',
    conn
)
patient_concussions[~pd.isna(patient_concussions['DEATHDATE'])].sum(0)

Id            39e76039-522c-61f1-d961-e03dce5f0bb20d679725-4...
BIRTHDATE     1998-11-211973-04-201960-12-271973-04-201952-0...
DEATHDATE     2003-07-172005-08-192000-06-212011-12-161955-0...
CONCUSSION                                                   14
dtype: object

In [27]:
patient_concussions = pd.read_sql(
    '''
    SELECT p.Id, p.birthdate, p.deathdate, c.DESCRIPTION, c.START
    FROM patients p
    JOIN conditions c on p.Id = c.patient
    ''',
    conn
)
patient_concussions[~pd.isna(patient_concussions['DEATHDATE'])].to_csv('../data/interim/scenario1/death_conditions.csv')